# State

In [4]:
from typing import Annotated, Literal, Optional
from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


def update_dialog_stack(left: list[str], right: Optional[str]) -> list[str]:
    """Push or pop the state."""
    if right is None:
        return left
    if right == "pop":
        return left[:-1]
    return left + [right]


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_info: str
    dialog_state: Annotated[
            list[
                Literal[
                    "assistant",
                    "query",
                    "requset",
                ]
            ],
            update_dialog_stack,
        ]

# Tools

In [12]:
from langchain_core.tools import tool
from django.core.exceptions import ObjectDoesNotExist

from products.models import Order, OrderItem

@tool
def order():
    """
    Places a new order.
    This function processes a new order request and confirms
    """
    return "주문 완료"

@tool
def change_order():
    """
    Modifies an existing order.
    This function allows the user to change details of an existing
    """
    return "주문 변경 완료"

@tool
def cancel_order():
    """
    Cancels an existing order.
    This function processes the cancellation of an existing order
    """
    return "주문 취소 완료"

@tool
def view_order(state: State):
    """
    Views the details of a specific order.
    This function retrieves and returns the details of an order.
    """
    user_id = state["user_info"]

    try:
        orders = Order.objects.filter(user_id=user_id).order_by('-created_at')[:5]
        recent_orders = []
        for order in orders:
            order_items = OrderItem.objects.filter(order=order)
            items_details = [
                {
                    "product_name": item.product.product_name,
                    "quantity": item.quantity,
                    "price": float(item.price)  # Decimal을 float으로 변환
                } for item in order_items
            ]
            recent_orders.append({
                "id": order.id,
                "created_at": order.created_at.isoformat(),
                "order_status": order.order_status,
                "items": items_details
            })
        
        return recent_orders
    except ObjectDoesNotExist:
        return []
    
    # return "주문 조회 완료"

@tool
def view_change_order():
    """
    Views the details of a modified order.
    This function retrieves and returns the details of the changes made to an order.
    """
    return "주문 변경 조회 완료"

@tool
def view_cancel_order():
    """
    Views the details of a canceled order.
    This function retrieves and returns the details of a canceled order.
    """
    return "주문 취소 조회 완료"

ModuleNotFoundError: No module named 'products'

In [6]:
from langchain_core.pydantic_v1 import BaseModel, Field


class CompleteOrEscalate(BaseModel):
    """A tool to mark the current task as completed and/or to escalate control of the dialog to the main assistant,
    who can re-route the dialog based on the user's needs."""

    cancel: bool = True
    reason: str

    class Config:
        schema_extra = {
            "example": {
                "cancel": True,
                "reason": "User changed their mind about the current task.",
            },
            "example 2": {
                "cancel": True,
                "reason": "I have fully completed the task.",
            },
            "example 3": {
                "cancel": False,
                "reason": "I need to use the tool to place a new order.",
            },
        }

# order_request.py로 옮길 것을 제안
class ToOrderInquiryAssistant(BaseModel):
    """Transfers work to a specialized assistant to handle order queries."""

    order_id: int = Field(description="The ID of the order to query.")
    request: str = Field(description="Any necessary follow-up questions the querying assistant should clarify before proceeding.")

class ToOrderRequestAssistant(BaseModel):
    """Transfers work to a specialized assistant to handle order placements, modifications, or cancellations."""

    order_id: int = Field(description="The ID of the order to update.")
    action: str = Field(description="The action to perform: 'order', 'change_order', 'cancel_order'.")
    request: str = Field(description="Any additional information or requests from the user regarding the order.")

# Assistant

In [7]:
from langchain_openai import ChatOpenAI

model="gpt-3.5-turbo"
# model="gpt-4o"
# model="gpt-4-turbo-2024-04-09"
llm = ChatOpenAI(model=model)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001AD3E30FD00>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001AD3E30FA60>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [8]:
import json

from django.db.models import QuerySet
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_core.messages import AIMessage

class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            result = self.runnable.invoke(state)

            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        if isinstance(result, Order):
            result = result.to_dict()
            result = AIMessage(content=result)
            # result = json.dumps(result)
        elif isinstance(result, QuerySet):
            result = json.dumps([order.to_dict() for order in result])
            result = AIMessage(content=result)

        return {"messages": result}

In [9]:
from datetime import datetime

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_inquiry_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a specialized assistant for handling order queries. "
            "The primary assistant delegates work to you whenever the user needs help with their orders. "
            "Confirm the order details with the customer and inform them of any additional information. "
            "If you need more information or the customer changes their mind, escalate the task back to the main assistant."
            "\n\nCurrent user order information:\n<Orders>\n{user_info}\n</Orders>"
            "\nCurrent time: {time}."
            "\n\nIf the user needs help, and none of your tools are appropriate for it, then"
            '"CompleteOrEscalate" the dialog to the host assistant. Do not waste the user\'s time. Do not make up invalid tools or functions.',
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
).partial(time=datetime.now())

inquiry_tools = [view_order, view_change_order, view_cancel_order]
order_inquiry_runnable = (
    order_inquiry_prompt 
    | llm.bind_tools(inquiry_tools + [CompleteOrEscalate])
)
order_inquiry_runnable

NameError: name 'view_order' is not defined